In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import os
import pdb

In [11]:
def get_model(maxlen, num_chars, num_layers, num_units):
    print('Build model...')
    model = Sequential()
    for layer_idx in range(num_layers):
        if layer_idx == 0:
            #the size is maxlen x numchars. Maxlen is because of the length of a sentence (128 words) while num_chars=119
            #which are the unique words in my vocabulary.
            #NB: the input is one hot encoded over the vocabulary size (num_chars)
            model.add(LSTM(num_units, return_sequences=True, input_shape=(maxlen, num_chars)))
        else:
            model.add(LSTM(num_units, return_sequences=False))
        model.add(Dropout(0.2))

    model.add(Dense(num_chars))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [12]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    #a = np.asarray(a).astype('float64')

    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    print('printo a')
    print(a)
    print('printo il valore del random')
    print(np.argmax(np.random.multinomial(1, a, 1)))
    return np.argmax(np.random.multinomial(1, a, 1))

In [13]:
def run(is_character=False, maxlen=None, num_units=None, model_prefix=''):

    character_mode = is_character

    if character_mode:
        if maxlen == None:
            maxlen = 1024
        if num_units == None:
            num_units = 32
        step = 2*17 # step to create training data for truncated-BPTT
    else: # word mode
        if maxlen == None:
            maxlen = 128 # maxlength used in RNN input
        if num_units == None: 
            num_units = 512 #number of unit per layer LSTM 512 
        step = 8

    if character_mode:
        num_char_pred = maxlen*3/2
    else: 
        num_char_pred = 17*30 #this should be the number of elements predicted in the output. How "long" is my output sequence

    num_layers = 2
    # 
    if character_mode:
        prefix = 'char'
    else:
        prefix = 'word'

    path = 'metallica_drums_text.txt' # Corpus file
    text = open(path).read()
    print('corpus length:', len(text))

    if character_mode:
        chars = set(text)
    else:
        chord_seq = text.split(' ')
        chars = set(chord_seq) #contains the unique words in my dictionary. They are 119
        text = chord_seq #contains the full text in an array format. Each entry of my array is a word of type 0xb0110101010 

    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    num_chars = len(char_indices) #number of unique words in my training set
    print('total chars:', num_chars)

    # cut the text in semi-redundant sequences of maxlen characters

    sentences = []
    next_chars = []
    #Here im creating the inputs and targets for my RNN. Each single input has length maxlen.
    #Inputs are semi-redundant, in the sense that i take a length of maxlen=128 and the step is 8. So my first part of the input
    #will be the same and the last 8 elements are "new". I'm just "slitting" of 8 notes ahead
    for i in range(0, len(text) - maxlen, step): #iterates over the range with steps of 8.
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    print('nb sequences:', len(sentences))
    print('Vectorization...')
    
    #Here i'm creating the input dataset and target dataset for my network
    #X is a tri-dimensional vector: 1 dimension -> Sentences, 2 dimension -> Single sentence, 3 dimension -> one hot encoded vector of the single word
    #So basically i have a structure where i have N sentences of maxlen Words where each word is represented as a one hot vector of length num_chars
    X = np.zeros((len(sentences), maxlen, num_chars), dtype=np.bool) #Input matrix
    y = np.zeros((len(sentences), num_chars), dtype=np.bool) #Target Matrix
    #Here i'm actually "populating" the matrixes, which were initialized with all zeros
    print('Entering initialization cycle')

    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1 #NB: char in this case means a whole word like oxb01011101. With char_indices[char] i'm retrieving the index of my word inside my dictionary of (words,index)
        #print('Finished input initialization')
        y[i, char_indices[next_chars[i]]] = 1
    print('Completed Vectorization')
    # build the model: 2 stacked LSTM
    model = get_model(maxlen, num_chars, num_layers, num_units) 
    
    #Just some string declarations for folders management and names.
    #NB: CHANGE THE / with \ for windows! 
    result_directory = 'r_%s_%s_%d_%d_units' % (prefix, model_prefix, maxlen, num_units)
    filepath_model = os.path.join(result_directory, 'best_model.hdf')

    #filepath_model = '%sbest_model.hdf' % result_directory
    description_model = '%s, %d layers, %d units, %d maxlen, %d steps' % (prefix, num_layers, num_units, maxlen, step)
    
    #Usual Model checkpoints and Early Stopping
    checker = tf.keras.callbacks.ModelCheckpoint(filepath_model, monitor='loss', verbose=0, save_best_only=True, mode='auto')
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15, verbose=0, mode='auto')
    
    #create a result directory if it doesn't exist
    if not os.path.exists(result_directory):
        os.mkdir(result_directory)

    # write a description file.
    #creates an empty file with the drscription of my model as title
    with open(result_directory+description_model, 'w') as f_description:
        pass

    # train the model, output generated text after each iteration
    batch_size = 128 #Size of a training batch. So basically i'll update my loss function every 128 input sentences (usual batch gradient descent)
    loss_history = []
    pt_x = [1]
    #An epoch is a complete iteration over the whole input training set. So 10 epochs means that i iterates 10 times over my input dataset
    nb_epochs = [np.sum(pt_x[:i+1]) for i in range(len(pt_x))] #array containing many epochs length. The model will be fitted many times, one for each nb_epochs.

    # not random seed, but the same seed for all.
    #A random seed (or seed state, or just seed) is a number (or vector) used to initialize a pseudorandom number generator.
    start_index = random.randint(0, len(text) - maxlen - 1)

    for iteration, nb_epoch in zip(pt_x,nb_epochs):
        if os.path.exists('stop_asap.keunwoo'):
            os.remove('stop_asap.keunwoo')
            break

        print('-' * 50)
        print('Iteration', iteration)
        
        #fitting model over nb_epochs
        #result = model.fit(X, y, batch_size=batch_size, nb_epoch=nb_epoch, callbacks=[checker, early_stop]) 
        #loss_history = loss_history + result.history['loss']

        print ('Saving model after %d epochs...' % nb_epoch)
        #Saving model weights. Saving a model trained over nb_epochs
        #model.save_weights('%smodel_after_%d.hdf'%(result_directory, nb_epoch), overwrite=True) 
        model.load_weights("model_weights.h5")
        w2 = model.get_weights()
        print(w2)

        
        for diversity in [0.9, 1.0, 1.2]:
            #creates a .txt file where i will save my predictions
            with open(('%sresult_%s_iter_%02d_diversity_%4.2f.txt' % (result_directory, prefix, iteration, diversity)), 'w') as f_write:

                print()
                print('----- diversity:', diversity)
                f_write.write('diversity:%4.2f\n' % diversity)
                if character_mode:
                    generated = ''
                else:
                    generated = [] #simple initialization
                #selects a random sentence from my input dataset.
                sentence = text[start_index: start_index + maxlen]
                seed_sentence = text[start_index: start_index + maxlen]

                if character_mode:
                    generated += sentence
                else:
                    #at first iteration i just add my input sentence in my generated element
                    generated = generated + sentence


                print('----- Generating with seed:')

                if character_mode:
                    print(sentence)
                    sys.stdout.write(generated)
                else:
                    print(' '.join(sentence))

                for i in range(num_char_pred): 
                    # if generated.endswith('_END_'):
                    # 	break
                    x = np.zeros((1, maxlen, num_chars)) #initialization of input. Matrix of maxlen words, each 

                    for t, char in enumerate(sentence):
                        x[0, t, char_indices[char]] = 1. 

                    preds = model.predict(x, verbose=0)[0] 
                    #print('printo la prediction')
                    #print(preds)
                    next_index = sample(preds, diversity)
                    next_char = indices_char[next_index]
                    #print('printo il next char')
                    #print(next_char)

                    if character_mode:
                        generated += next_char
                        sentence = sentence[1:] + next_char
                    else:
                        generated.append(next_char)
                        sentence = sentence[1:]
                        sentence.append(next_char)
                        #print('printo la sentence generata')
                        #print(generated)

                    if character_mode:
                        sys.stdout.write(next_char)
                    # else:
                    # 	for ch in next_char:
                    # 		sys.stdout.write(ch)	

                    sys.stdout.flush()

                if character_mode:
                    f_write.write(seed_sentence + '\n')
                    f_write.write(generated)
                else:
                    f_write.write(' '.join(seed_sentence) + '\n')
                    
                    f_write.write(' ' .join(generated))

        np.save('%sloss_%s.npy'%(result_directory, prefix), loss_history)

    print ('Done! You might want to run main_post_process.py to get midi files. ')
    print ('You need python-midi (https://github.com/vishnubob/python-midi) to run it.')


In [14]:
run()

corpus length: 2141692
total chars: 119
nb sequences: 23204
Vectorization...
Entering initialization cycle
Completed Vectorization
Build model...
--------------------------------------------------
Iteration 5
Saving model after 5 epochs...
[array([[ 0.02269091, -0.04516821, -0.01258686, ..., -0.01171814,
        -0.01153291, -0.01094408],
       [ 0.13398412, -0.06601574,  0.02606351, ..., -0.11501969,
         0.07536041, -0.05763862],
       [-0.10696831,  0.1161921 ,  0.00254606, ..., -0.03345156,
        -0.04903811,  0.10501978],
       ...,
       [ 0.09328523,  0.01390448,  0.0592925 , ...,  0.05467119,
        -0.11179421, -0.16174896],
       [ 0.02263589, -0.0035955 ,  0.07794168, ..., -0.06655729,
         0.00791793, -0.09032465],
       [-0.05744762,  0.00170897, -0.04038751, ..., -0.23021223,
         0.02558188,  0.17588174]], dtype=float32), array([[-0.01958634, -0.02770473,  0.0586572 , ...,  0.10881726,
         0.21123828,  0.03784214],
       [ 0.04006571,  0.009756

printo a
[1.32301182e-06 1.89423065e-06 1.46966192e-06 1.93022265e-06
 2.19168282e-06 3.76839447e-03 8.64907452e-06 1.15343591e-05
 2.92368895e-06 3.04026548e-06 2.52268859e-03 2.53181338e-06
 1.26800966e-04 8.61838832e-02 1.30930289e-06 1.05776172e-03
 1.48073115e-04 1.12871214e-06 1.38728498e-02 5.23081951e-07
 3.85236053e-04 4.98314761e-03 5.42087510e-05 2.32915477e-06
 1.26721784e-06 7.06205843e-03 1.54172685e-02 2.61253786e-06
 2.38113061e-01 1.50833978e-06 1.79263668e-06 2.45944193e-06
 8.29317578e-05 1.81422452e-04 3.23561767e-06 1.95673609e-04
 4.35100694e-04 1.07508868e-05 1.92318368e-03 1.35150243e-04
 1.45257445e-06 1.67802423e-02 6.23928558e-04 7.76135494e-06
 2.39197016e-05 5.04068021e-06 3.05331491e-06 3.32573429e-04
 1.90512985e-02 1.85379520e-01 2.67241616e-03 2.35005450e-06
 6.28356247e-06 1.05889449e-05 3.47866489e-06 4.31425360e-06
 8.97176284e-03 1.11126038e-03 2.54183169e-03 3.50057599e-06
 7.09695043e-03 2.79349566e-01 1.14642293e-03 3.80177130e-06
 4.32641627e-06

printo a
[1.10510030e-07 9.85761517e-08 9.86416069e-08 1.22505099e-07
 1.19575787e-07 3.30032162e-05 1.19866243e-07 6.41295919e-05
 2.42750588e-07 2.29202769e-07 1.13969028e-03 1.38345712e-07
 8.77693601e-05 2.89518982e-01 8.61835687e-08 1.02066326e-04
 2.51390302e-05 1.24005439e-07 8.86574015e-03 6.42549338e-08
 4.33719833e-05 7.72825442e-03 3.80410461e-06 1.27800092e-07
 1.14776221e-07 1.82393589e-03 3.81710986e-03 1.51929271e-07
 2.47064829e-01 2.31022909e-07 1.29041695e-07 2.20247742e-07
 2.42923920e-06 6.32876618e-06 2.43982043e-07 3.83276711e-06
 7.57367088e-05 1.30131878e-06 1.38633710e-04 3.97761369e-06
 8.68330119e-08 2.57249758e-05 2.77549698e-05 4.24377106e-07
 4.67953942e-06 2.51510443e-07 1.63802270e-07 2.45475559e-03
 3.32804263e-01 2.37706192e-02 8.26483883e-04 1.97738203e-07
 2.64364331e-07 4.44413985e-07 3.63701076e-07 1.59901774e-07
 3.76951532e-04 1.75455949e-04 1.12376920e-05 1.42961923e-07
 5.48801338e-03 9.10153333e-03 5.04158146e-04 2.32121820e-07
 2.13803133e-07

printo a
[6.03775803e-08 4.46444659e-08 3.56511514e-08 4.71272479e-08
 1.30576979e-07 1.67466824e-05 2.44246280e-07 1.35567043e-05
 1.19536097e-07 1.25139977e-07 2.12669602e-05 9.94139313e-08
 1.49080661e-05 2.32739925e-01 3.14225090e-08 7.09397136e-05
 6.54536143e-06 4.93576664e-08 5.29296522e-04 2.58373962e-08
 9.33188130e-06 4.69373888e-04 1.53872475e-06 6.73540015e-08
 5.35537410e-08 1.67908132e-04 9.75737465e-04 1.37223537e-07
 6.16383612e-01 4.99524369e-08 6.95502607e-08 9.68699609e-08
 5.03323145e-06 6.44692136e-06 9.49407166e-08 2.29754369e-06
 1.56455790e-04 1.27967235e-06 6.17694750e-05 5.70778411e-06
 1.39105829e-08 2.60216111e-05 6.51532755e-05 5.60984518e-07
 3.69677855e-06 1.89200605e-07 1.19215969e-07 4.13803864e-05
 2.71202112e-03 1.17309473e-01 4.91251703e-04 1.19673430e-07
 2.30960978e-07 3.40542243e-07 1.48849239e-07 1.46183098e-07
 2.69004086e-04 5.16426051e-04 1.32869081e-06 1.08516112e-07
 6.15450088e-03 7.16290437e-04 9.17669604e-06 4.94589074e-07
 1.48615200e-07

printo a
[1.02058777e-07 8.32892013e-08 1.00170524e-07 1.10722489e-07
 1.95923405e-07 5.07894547e-05 2.05697958e-07 2.62963040e-05
 2.36173733e-07 2.49961175e-07 3.35050601e-04 1.83282310e-07
 8.56743936e-05 3.89858820e-02 9.17863971e-08 3.32213764e-04
 3.04435634e-05 1.19864424e-07 2.62016058e-03 4.67983341e-08
 5.89569281e-05 2.32186937e-03 6.20665833e-06 1.47667080e-07
 7.82378891e-08 5.11722581e-04 5.91198448e-03 1.82228206e-07
 7.39635408e-01 1.70663156e-07 9.41925435e-08 2.13259412e-07
 1.05772915e-05 2.03897616e-05 2.48533780e-07 1.79081162e-05
 9.53812923e-05 1.43068507e-06 1.07859261e-04 1.60042910e-05
 6.46643485e-08 2.95943064e-05 8.48556374e-05 7.21317235e-07
 2.30709929e-06 3.36361126e-07 2.12695767e-07 5.08836063e-04
 5.49828680e-03 1.48682997e-01 7.81919924e-04 2.75212813e-07
 4.06035554e-07 5.03524575e-07 3.89458933e-07 1.74561450e-07
 6.63385028e-04 3.98638833e-04 4.34006370e-06 2.22646818e-07
 8.50103144e-03 6.85863663e-03 1.38013973e-04 6.15477916e-07
 3.37353811e-07

...

printo a
[2.17067591e-08 2.25460273e-08 2.06685336e-08 2.19488960e-08
 5.74278722e-08 5.13750228e-06 5.83380171e-08 3.35912919e-05
 3.99507378e-08 4.29092282e-08 6.04350425e-05 3.81206782e-08
 2.74087288e-05 7.75037287e-03 2.03415009e-08 1.54907990e-04
 3.14716931e-06 3.11004946e-08 5.53149264e-04 1.55734821e-08
 1.47822757e-05 1.59487512e-03 1.24661415e-06 3.91653963e-08
 1.48222750e-08 1.27643987e-04 9.00991959e-04 3.36471579e-08
 9.31435108e-01 3.19842215e-08 2.36665070e-08 4.06853502e-08
 2.99269755e-06 4.46018203e-06 4.75681681e-08 1.11214467e-05
 2.36215383e-05 5.40501276e-07 1.09062867e-05 4.34105596e-06
 1.47747699e-08 7.45164414e-07 1.62149936e-05 1.10811307e-07
 2.78680977e-06 5.32617328e-08 3.08068344e-08 1.39646392e-04
 3.45341209e-03 2.00641565e-02 1.49476051e-04 3.49359617e-08
 7.44802335e-08 1.02486439e-07 7.43710729e-08 4.48015420e-08
 7.42378979e-05 1.84429620e-04 5.41886493e-07 5.50675416e-08
 1.99880451e-03 3.71724763e-03 9.89759792e-05 5.66837684e-07
 4.91371566e-08

printo a
[2.09976019e-08 2.17634941e-08 1.95356282e-08 2.10487681e-08
 5.79645167e-08 4.57439046e-06 5.96581131e-08 3.28414244e-05
 3.74545586e-08 4.00513400e-08 4.70509331e-05 3.81143224e-08
 2.33883038e-05 7.10838614e-03 1.86385023e-08 1.49453263e-04
 2.53834651e-06 2.91314954e-08 4.90999955e-04 1.47550443e-08
 1.25783208e-05 1.47834281e-03 1.16216859e-06 3.83445915e-08
 1.43216612e-08 1.21922516e-04 7.69304868e-04 3.38202746e-08
 9.34985697e-01 2.86994446e-08 2.31107613e-08 4.08157419e-08
 3.00026772e-06 4.02777141e-06 4.46259030e-08 1.10602286e-05
 2.29157959e-05 5.18886736e-07 9.55726318e-06 4.14297619e-06
 1.34550069e-08 6.68518112e-07 1.50903879e-05 1.12519750e-07
 3.21844595e-06 5.17350713e-08 2.97405887e-08 1.10892681e-04
 3.16460500e-03 1.90221928e-02 1.27044666e-04 3.06025960e-08
 7.27134974e-08 9.98951961e-08 6.87836490e-08 4.65519960e-08
 6.36421173e-05 1.85787168e-04 5.35200229e-07 5.35977840e-08
 1.65691006e-03 3.33854184e-03 8.92137832e-05 6.00463352e-07
 4.71643844e-08

printo a
[1.96779144e-08 2.01934185e-08 1.83517361e-08 1.85558449e-08
 4.93005317e-08 3.79750190e-06 5.91587188e-08 5.91291428e-05
 3.69036215e-08 3.69458100e-08 5.32456470e-05 3.43172886e-08
 2.40667559e-05 9.41202603e-03 1.65204774e-08 1.18859622e-04
 1.88212596e-06 2.82990360e-08 4.95155167e-04 1.58261617e-08
 1.00500692e-05 1.69256923e-03 8.62478601e-07 3.39238220e-08
 1.44294656e-08 9.63291968e-05 6.04415080e-04 2.80531154e-08
 9.24948573e-01 2.81292003e-08 2.26761117e-08 3.76451936e-08
 2.48564584e-06 3.64524362e-06 4.17105390e-08 8.84198653e-06
 2.09708087e-05 5.41923555e-07 7.03157957e-06 2.74538843e-06
 1.26143229e-08 3.28258466e-07 9.84526559e-06 1.01394946e-07
 3.64525067e-06 4.58514151e-08 2.48948648e-08 1.59828429e-04
 7.46661937e-03 1.71894711e-02 1.12697919e-04 2.68423488e-08
 6.19923668e-08 9.29604482e-08 5.95765535e-08 3.91461512e-08
 5.06390097e-05 1.68391838e-04 4.44460341e-07 4.61345877e-08
 1.69167609e-03 2.87750876e-03 9.75456860e-05 6.64998879e-07
 4.18263753e-08

printo a
[1.76926065e-08 1.75324679e-08 1.62645044e-08 1.81292830e-08
 4.99519288e-08 3.36791504e-06 4.86551137e-08 4.06182080e-05
 3.16517905e-08 3.39647990e-08 3.74697738e-05 3.12535136e-08
 2.10460985e-05 9.02724639e-03 1.52193458e-08 1.58086928e-04
 2.04025355e-06 2.22887682e-08 4.02158825e-04 1.31421123e-08
 9.30817077e-06 1.68040022e-03 1.23471796e-06 3.03250687e-08
 1.32045495e-08 1.06330786e-04 8.24079267e-04 2.83251698e-08
 9.25865233e-01 2.43440219e-08 2.03872279e-08 3.70396478e-08
 2.37492486e-06 3.61402772e-06 3.80285385e-08 1.15520352e-05
 2.44769381e-05 4.72763361e-07 7.84183976e-06 3.99844566e-06
 9.85487780e-09 4.71158302e-07 1.26553932e-05 9.65016298e-08
 2.57015290e-06 4.29830500e-08 2.40266083e-08 9.55623182e-05
 5.65225026e-03 1.97259206e-02 1.17691619e-04 2.35724542e-08
 5.62198039e-08 8.27233109e-08 5.80464672e-08 4.11036041e-08
 5.11631224e-05 1.63139062e-04 3.50580194e-07 4.45121380e-08
 1.82616676e-03 2.60018627e-03 7.66441153e-05 5.54344354e-07
 4.09361718e-08

KeyboardInterrupt: 

In [7]:
model.load_weights("model_weights.h5")

NameError: name 'model' is not defined